# bills_webscraper.ipynb

### Import python packages

In [1]:
import warnings
warnings.filterwarnings('ignore')

import re, sqlite3, sys
from collections import OrderedDict

import requests
from bs4 import BeautifulSoup
import selenium.webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException

### Define Bill class

In [2]:
class Bill:
    
    def __init__(self, num, link, congress, short_title, long_title, date_filed, scope, status, author,
                 subject, pri_committee, date_lastUpdate, logs, ra):
        self.num = num
        self.link = link
        self.congress = congress
        self.short_title = short_title
        self.long_title = long_title
        self.date_filed = date_filed
        self.scope = scope
        self.status = status
        self.author = author
        self.subject = subject
        self.pri_committee = pri_committee
        self.date_lastUpdate = date_lastUpdate
        self.logs = logs
        self.ra = ra
        self.entities = (num, link, congress, short_title, long_title, date_filed, scope, status, author,
                 subject, pri_committee, date_lastUpdate, logs, ra)
        
    def __str__(self):
        return 'Congress: ' + str(self.congress) + ' ' + self.num + ': ' + self.short_title
    
    def insert_bill():
        pass
    
    def update_bill():
        pass
    
    def remove_bill():
        pass

### Compile Regex Patterns
These regex patterns will be used to extract relevant strings only from the webscraping function.

In [5]:
bill_number_regex = re.compile(r'(?<=\d\d&q=).*')
filed_date_regex = re.compile(r'(?<=Filed on ).*(?= by )')
author_regex = re.compile(r'(?<= by ).*(?=\n)')
status_regex = re.compile(r'.*(?= \()')
last_update_regex = re.compile(r'(?<=\().*(?=\))')
logs_regex = re.compile(r'(?<=\n\[ \d\d\d\d \]\n).*(?=\(The (legislative history)|(LEGISLATIVE HISTORY))',re.DOTALL)
logs_wFootNote_regex = re.compile(r'(?<=, \d\d\w\w CONGRESS \]\n\[ \d\d\d\d \]\n).*(?=;)',re.DOTALL)
ra_regex = re.compile(r'(?<=\nRepublic Act No\. )\d*(?=(\n)| )',re.DOTALL)

In [3]:
def bills_logScraper(logs):
    try:
        return logs_regex.search(logs).group()
    except AttributeError:
        return logs_wFootNote_regex.search(logs).group()

#### getBill function
Our main scraping function.

In [6]:
driver = selenium.webdriver.PhantomJS()

link = 'http://www.senate.gov.ph/lis/bill_res.aspx?congress=17&q=SBN-1075'

print("Trying to access child link "+ link + ' ...')
driver.get(link)
driver.implicitly_wait(100)
try:
    allInfo_btn = driver.find_element_by_xpath("//a[@id='lbAll']")
except NoSuchElementException:
    print('\t' + str(num) + ' is missing! Skipping...\n')
    fetch_errors.append(num)
    pass

allInfo_btn.click()
element = WebDriverWait(driver, 
                        10).until(lambda x:x.find_element_by_xpath("//p[contains(text(),'Legislative History')]")) 
soup_allinfo = BeautifulSoup(driver.page_source, 'html.parser')

short_title = soup_allinfo.select('p[class="h1_bold"]')[0].getText()
allinfo_frame_text = soup_allinfo.select('td[id="content"]')[0].getText()
date_filed = filed_date_regex.search(allinfo_frame_text).group()
author = author_regex.search(allinfo_frame_text).group()
long_title = soup_allinfo.select('blockquote')[0].getText()
scope = soup_allinfo.select('blockquote')[1].getText()
legis_status = soup_allinfo.select('blockquote')[2].getText()
status = status_regex.search(legis_status).group()
date_lastUpdate = last_update_regex.search(legis_status).group()

# Special case if a bill was 'Withdrawn.'
if 'Withdrawn' in status:
    subject, pri_committee = ('','')
else: 
    subject = soup_allinfo.select('blockquote')[3].getText()       
    pri_committee = soup_allinfo.select('blockquote')[4].getText()

all_blkquotes = '\n'.join([soup_allinfo.select('blockquote')[i].
                   getText() for i in range(len(soup_allinfo.select('blockquote')))])        
logs = bills_logScraper(all_blkquotes)

try:
    ra = ra_regex.search(all_blkquotes).group()
except AttributeError:
    ra = ''


print('All Bills have been successfully scraped on this page.')
# Must close the ghostDriver before exiting this function...
driver.quit()

Trying to access child link http://www.senate.gov.ph/lis/bill_res.aspx?congress=17&q=SBN-1075 ...
All Bills have been successfully scraped on this page.


In [7]:
logs

'8/25/2016\nIntroduced by Senator JOSEPH VICTOR G. EJERCITO;\n8/30/2016\nRead on First Reading and Referred to the Committee on HEALTH AND DEMOGRAPHY;\n'